# Apriori算法

In [1]:
import numpy as np

## Apriori算法的一般过程
1. 收集数据：使用任意方法。
2. 准备数据：任何数据类型都可以，因为我们只保存集合。
3. 分析数据：使用任意方法。
4. 训练算法：使用Apriori算法来找到频繁项集。
5. 测试算法：不需要测试过程。
6. 使用算法：用于发现频繁项集以及物品之间的关联规则。

## 生成候选项集
**数据集扫描伪代码**  
对数据集中的每条交易记录tran  
对每个候选项集can：  
&emsp;&emsp;检查一下can是否是tran的子集：  
&emsp;&emsp;如果是，则增加can的计数值  
对每个候选项集：  
如果其支持度不低于最小值，则保留该项集  
返回所有频繁项集列表

### Apriori算法中的辅助函数

In [2]:
# 加载数据集
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

# 构建集合C1，大小为1的所有候选项集的集合。
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    # 对C1中每个项构建一个不变集合,
    # frozenset是冻结的集合，它是不可变的，存在哈希值，
    # 好处是它可以作为字典的key，也可以作为其它集合的元素。缺点是一旦创建便不能更改，没有add，remove方法。
    return list(map(frozenset, C1))   

# D：数据集，Ck：候选项集列表, minSupport：最小支持度
# 用于从C1生成L1，返回一个包含支持度值的字典
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                # 判断C1中的集合是记录的一部分
                if can not in ssCnt: ssCnt[can]=1
                # 增加字典中对应的计数值
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        # 计算支持度
        support = ssCnt[key]/numItems
        if support >= minSupport:
            #满足最小支持度要求，在列表的首部插入任意新的集合
            retList.insert(0,key)
        # 返回最频繁项集的支持度
        supportData[key] = support
    return retList, supportData

### 测试辅助函数

In [3]:
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [4]:
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [5]:
D = list(map(set, dataSet))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [6]:
L1, suppData0 = scanD(D, C1, 0.5)
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

## 组织完整的Apriori算法

**整个Apriori算法的伪代码**  
当集合中项的个数大于0时  
&emsp;&emsp;构建一个k个项组成的候选项集的列表  
&emsp;&emsp;检查数据以确认每个项集都是频繁的  
&emsp;&emsp;保留频繁项集并构建k+1项组成的候选项集的列表

In [7]:
# Lk：频繁项集列表, k：项集元素个数
# 输出Ck
def aprioriGen(Lk, k): 
    # 创建一个空列表
    retList = []
    # 计算Lk中的元素数目
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            # 取两个集合前面k-2个元素
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            # 进行排序
            L1.sort(); L2.sort()
            if L1==L2: 
                # 如果前面k-2个元素都相等，那么就将这两个集合合成一个大小为k的集合
                retList.append(Lk[i] | Lk[j])
    return retList

# dataSet：数据集，minSupport：最小支持度
def apriori(dataSet, minSupport = 0.5):
    # 创建C1
    C1 = createC1(dataSet)
    # 读入数据集将其转化为D（集合列表）
    D = list(map(set, dataSet))
    # 创建L1频繁项集
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    # 设定项集元素
    k = 2
    while (len(L[k-2]) > 0):
        # 得到Ck，创建包含更大项集的更大列表
        Ck = aprioriGen(L[k-2], k)
        # 得到Lk, 丢掉不满足最小支持度要求的项集
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        # Lk列表被添加到L
        L.append(Lk)
        # 增加k的值，
        k += 1
    return L, supportData

In [8]:
L, suppData = apriori(dataSet)
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [9]:
L[0]

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [10]:
L[1]

[frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})]

In [11]:
L[2]

[frozenset({2, 3, 5})]

In [12]:
L[3]

[]

In [13]:
aprioriGen(L[0], 2)

[frozenset({2, 5}),
 frozenset({3, 5}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({1, 3})]

这里有6个集合是候选项集Ck中的元素，其中4个集合在L[1]中，剩下2个元素被函数scanD()过滤掉。

尝试一下70%的支持度

In [14]:
L, suppData = apriori(dataSet, minSupport=0.7)
L

[[frozenset({5}), frozenset({2}), frozenset({3})], [frozenset({2, 5})], []]

&emsp;&emsp;可以利用关联规则的上述性质属性来减少需要测试的规则数目。可以首先从一个频繁项集开始，接着创建一个规则列表，其中规则右部只包含一个元素，然后对这些规则进行测试。接下来合并所有剩余规则来创建一个新的规则列表，其中规则右部包含两个元素。这种方法也被称作为**分级法**。

## 从频繁项集中挖掘关联规则

In [15]:
# 主函数，L：频繁项集列表，supportData：包含那些频繁项集支持数据的字典，minConf：最新可信度阈值。
# bigRuleList：返回一个包含可信度的规则列表
def generateRules(L, supportData, minConf=0.7): 
    bigRuleList = []
    # 只获取有两个或更多元素的集合
    for i in range(1, len(L)):
        # freqSet：频繁项集
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                # 做进一步的合并
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                # 计算可信度
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

# 计算规则可信度以及找到满足最小可信度要求的规则
# brl: bigRuleList包含可信度的规则列表
# freqSet：频繁项集
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    # 满足最小可信度要求的规则列表
    prunedH = [] 
    # 遍历H
    for conseq in H:
        # 计算项集的可信度
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        # 找到满足最小的可信度
        if conf >= minConf: 
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            # 通过检查的规则保存起来
            prunedH.append(conseq)
    # 返回已通过检查的规则
    return prunedH

# 从最初的项集中生成更多的关联规则
# freqSet: 频繁项集，H：可以出现在规则右部的元素列表H
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    # 计算H中的频繁集大小
    m = len(H[0])
    # 查看该频繁项集是否大到可以移除大小为m的子集
    if (len(freqSet) > (m + 1)): 
        # 创建m+1项的更大的项集
        Hmp1 = aprioriGen(H, m+1)
        # 计算可信度。以确定是否满足要求
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):
            # 如果不止一条规则满足要求，进行合并
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [16]:
L, suppData = apriori(dataSet, minSupport=0.5)

In [17]:
rules = generateRules(L, suppData, minConf=0.7)
rules

frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({1}) --> frozenset({3}) conf: 1.0


[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]

In [18]:
rules = generateRules(L, suppData, minConf=0.5)
rules

frozenset({3}) --> frozenset({2}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({5}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({3}) --> frozenset({1}) conf: 0.6666666666666666
frozenset({1}) --> frozenset({3}) conf: 1.0
frozenset({5}) --> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf: 0.6666666666666666


[(frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]